# Example with spikeinterface NEW API

This is not based on actual spikeinterfacre but the futur one!!!

For installing, you need to do this in a fresh environement:
```
git clone https://github.com/samuelgarcia/spikeinterface.git
cd spikeinterface
python setup.py develop
```





In [1]:
# Import packages    
%matplotlib widget
import matplotlib.pyplot as plt
from pathlib import Path
import spikeinterface.full as si
from probeinterface import read_spikeglx
from probeinterface.plotting import plot_probe
import numpy as np

In [2]:
print(si.installed_sorters())

['herdingspikes', 'spykingcircus', 'tridesclous']


## read dataset

In [3]:
base_folder = Path('/home/arthur/Documents/SpikeSorting/Test_20210518/') 

data_folder = base_folder / 'raw_awake'

out_path = Path('/media/storage/spikesorting_output/REST/')

recording = si.SpikeGLXRecordingExtractor(data_folder)
print(recording)

probe = read_spikeglx(data_folder / 'raw_awake_01_g0_t0.imec0.ap.meta')
print(probe)
recording = recording.set_probe(probe)
print(recording)

SpikeGLXRecordingExtractor: 385 channels - 1 segments - 30.0kHz - 3881.358s
384
Probe - 384ch
ChannelSliceRecording: 384 channels - 1 segments - 30.0kHz - 3881.358s


In [5]:
fs = recording.get_sampling_frequency()

rest_window = (2000, 2500) # Time in seconds

rest_rec = recording.frame_slice(rest_window[0]*fs, rest_window[1]*fs)
rest_rec = rest_rec.set_probe(probe)
print(rest_rec)

FrameSliceRecording: 384 channels - 1 segments - 30.0kHz - 500.000s


In [ ]:
w_ts = si.plot_timeseries(rest_rec, time_range=(0,50), channel_ids=rest_rec.channel_ids[0:2]) # plotting just the first 10 channels
w_ts.figure.suptitle('Recording First 10 Channels')
w_ts.ax.set_y_label('Channels IDs')

## check probe

In [36]:
si.plot_probe_map(rest_rec, channel_ids=rest_rec.channel_ids[0:100]) # You can change which channels are plotted

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## run tridesclous

This is slow, run this once.

In [6]:
tdc_params = {'freq_min': 300.,
              'freq_max': 6000.,
              'detect_threshold' : 6, 
              'common_ref_removal': True, 
              'nested_params' : {'peak_detector': {'adjacency_radius_um': 100},
                                 'clean_peaks': {'alien_value_threshold': 100.}}
                            }
sorting_TCD = si.run_tridesclous(rest_rec, output_folder=out_path / 'output_TDC_new',
         verbose=True, **tdc_params)

Local copy of recording


write_binary_recording: 100%|██████████| 185/185 [00:57<00:00,  3.20it/s]


DataIO <id: 140508784883312> 
  workdir: /media/storage/spikesorting_output/REST/output_TDC_new
  sample_rate: 30000.482083
  total_channel: 384
  channel_groups: 0 [ch0 ch1 ch2 ch3 ... ch380 ch381 ch382 ch383]
  nb_segment: 1
  length: 15000241
  durations: 500.0 s.
catalogue_nested_params
{'chunksize': 30000,
 'clean_cluster': {'apply_auto_merge_cluster': True,
                   'apply_auto_split': True,
                   'apply_trash_low_extremum': True,
                   'apply_trash_not_aligned': True,
                   'apply_trash_small_cluster': True},
 'clean_peaks': {'alien_value_threshold': 100.0, 'mode': 'extremum_amplitude'},
 'cluster_kargs': {'adjacency_radius_um': 50.0,
                   'high_adjacency_radius_um': 30.0,
                   'max_loop': 7680,
                   'min_cluster_size': 20},
 'cluster_method': 'pruningshears',
 'duration': 499.9999986166889,
 'extract_waveforms': {'wf_left_long_ms': -2.5,
                       'wf_left_ms': -1.0,
        

## run herdingspikes -- not working

This is slow, run this once.

In [19]:
hs_params = {'clustering_bandwidth': 5.5,
             'clustering_alpha': 5.5, 
             'clustering_n_jobs': -1, 
             'clustering_bin_seeding': True, 
             'clustering_min_bin_freq': 16,
             'clustering_subset': None, 
             'left_cutout_time': 0.3, 
             'right_cutout_time': 1.8,
             'detect_threshold': 6, # default is 20
             'probe_masked_channels': [],
             'probe_inner_radius': 70, 
             'probe_neighbor_radius': 90, 
             'probe_event_length': 0.26,
             'probe_peak_jitter': 0.2, 
             't_inc': 100000, 
             'num_com_centers': 1, 
             'maa': 12, 
             'ahpthr': 11, 
             'out_file_name': 'HS2_detected', 
             'decay_filtering': False, 
             'save_all': False, 
             'amp_evaluation_time': 0.4, 
             'spk_evaluation_time': 1.0, 
             'pca_ncomponents': 2, 
             'pca_whiten': True, 
             'freq_min': 300.0, 
             'freq_max': 6000.0, 
             'filter': True, 
             'pre_scale': True, 
             'pre_scale_value': 20.0, 
             'filter_duplicates': True}

sorting_HS = si.run_herdingspikes(rest_rec, output_folder=out_path / 'output_herdingspikes', **hs_params)
print(sorting_HS)

Herdingspikes use the OLD spikeextractors with RecordingExtractorOldAPI
# Generating new position and neighbor files from data file
# Not Masking any Channels
# Sampling rate: 30000
# Localization On
# Number of recorded channels: 384
# Analysing frames: 15000241; Seconds: 500.00803333333334
# Frames before spike in cutout: 9
# Frames after spike in cutout: 54
# tcuts: 39 84
# tInc: 100000
# Analysing frames from -39 to 100084  (0.0%)
# Analysing frames from 99961 to 200084  (0.7%)
# Analysing frames from 199961 to 300084  (1.3%)
# Analysing frames from 299961 to 400084  (2.0%)
# Analysing frames from 399961 to 500084  (2.7%)
# Analysing frames from 499961 to 600084  (3.3%)
# Analysing frames from 599961 to 700084  (4.0%)
# Analysing frames from 699961 to 800084  (4.7%)
# Analysing frames from 799961 to 900084  (5.3%)
# Analysing frames from 899961 to 1000084  (6.0%)
# Analysing frames from 999961 to 1100084  (6.7%)
# Analysing frames from 1099961 to 1200084  (7.3%)
# Analysing frames 

# Analysing frames from 14899961 to 15000084  (99.3%)
# Analysing frames from 14999961 to 15000241  (100.0%)
# Detection completed, time taken: 0:03:40.369553
# Time per frame: 0:00:00.014691
# Time per sample: 0:00:00.000038
Loaded 0 spikes.
Traceback (most recent call last):
  File "/home/arthur/GitHub/spikeinterface/spikeinterface/sorters/basesorter.py", line 203, in run_from_folder
    SorterClass._run_from_folder(output_folder, sorter_params, verbose)
  File "/home/arthur/GitHub/spikeinterface/spikeinterface/sorters/herdingspikes/herdingspikes.py", line 214, in _run_from_folder
    uids = C.spikes.cl.unique()
  File "/home/arthur/Envs/spikeinterface/lib/python3.8/site-packages/pandas/core/generic.py", line 5465, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'DataFrame' object has no attribute 'cl'



/home/arthur/Envs/spikeinterface/lib/python3.8/site-packages/herdingspikes/hs2.py:161: UserWarning: Loading an empty file /media/storage/spikesorting_output/REST/output_herdingspikes/HS2_detected.bin . This usually happens when no spikes weredetected due to the detection parameters being set too strictly
  warnings.warn(


SpikeSortingError: Spike sorting failed. You can inspect the runtime trace in spikeinterface_log.json

## run spyking circus

This is slow, run this once.

In [7]:
sc_params = {'detect_sign': -1,
             'adjacency_radius': 100,
             'detect_threshold': 6,
             'template_width_ms': 3,
             'filter': True,
             'merge_spikes': True,
             'auto_merge': 0.75,
             'num_workers': None,
             'whitening_max_elts': 1000,
             'clustering_max_elts': 10000}
sorting_SC = si.run_spykingcircus(rest_rec, output_folder=out_path / 'output_spykingcircus_new', **sc_params)

RUNNING SHELL SCRIPT: /media/storage/spikesorting_output/REST/output_spykingcircus_new/run_spykingcircus.sh


## again sorting results

In [8]:
#sorting_HS = si.HerdingspikesSortingExtractor(out_path / 'output_herdingspikes')
#print(sorting_HS)

sorting_SC = si.SpykingCircusSortingExtractor(out_path / 'output_spykingcircus_new')
print(sorting_SC)

sorting_TDC = si.TridesclousSortingExtractor(out_path / 'output_TDC_new')
print(sorting_TDC)

SpykingCircusSortingExtractor: 325 units - 1 segments - 30.0kHz
TridesclousSortingExtractor: 15 units - 1 segments - 30.0kHz


## save sorting in an other formats

In [9]:
save_folder = out_path / 'save_sorting'
save_folder.mkdir(exist_ok=True)

In [10]:
#si.NpzSortingExtractor.write_sorting(sorting_HS, save_folder / 'HS.npz')
si.NpzSortingExtractor.write_sorting(sorting_SC, save_folder / 'SC_new.npz')
si.NpzSortingExtractor.write_sorting(sorting_TDC, save_folder / 'TDCS3_new.npz')

## Reimporting 

In [15]:
TDC = si.NpzSortingExtractor(save_folder / 'TDCS3_new.npz')
SC = si.NpzSortingExtractor(save_folder / 'SC_new.npz')

In [16]:
print(TDC)
print(SC)

NpzSortingExtractor: 15 units - 1 segments - 30.0kHz
  file_path: /media/storage/spikesorting_output/REST/save_sorting/TDCS3_new.npz
NpzSortingExtractor: 325 units - 1 segments - 30.0kHz
  file_path: /media/storage/spikesorting_output/REST/save_sorting/SC_new.npz


## Extracting waveforms to work with Phy Exporter

In [18]:
rest_rec_bp = si.bandpass_filter(rest_rec, freq_min=300, freq_max=6000) # apply filter
folder_TDC = out_path / 'Phy/TDC-REST'
folder_TDC_wv = folder_TDC / 'waveforms_filtered_new'
we_TDC = si.extract_waveforms(rest_rec_bp, TDC, folder_TDC_wv,
    load_if_exists=True, ms_before=1, ms_after=2., max_spikes_per_unit=500,
    chunk_size=30000, n_jobs=6, progress_bar=True)
print(we_TDC)

extract waveforms: 100%|██████████| 501/501 [00:33<00:00, 14.84it/s]

WaveformExtractor: 384 channels - 15 units - 1 segments
  before:30 after60 n_per_units: 500


In [19]:
folder_TDC_params = out_path / 'Phy/TDC-REST/phy-params_new'
si.export_to_phy(rest_rec_bp, TDC, folder_TDC_params, waveform_extractor=we_TDC,
            compute_pc_features=False, compute_amplitudes=True,
            remove_if_exists=True, chunk_size=30000, n_jobs=6, progress_bar=True)

You can run spikeinterface.toolkit.preprocessing.bandpass_filter(recording)
write_binary_recording with n_jobs 6  chunk_size 30000


extract amplitudes: 100%|██████████| 501/501 [00:32<00:00, 15.28it/s]

Run:
phy template-gui  /media/storage/spikesorting_output/REST/Phy/TDC-REST/phy-params_new/params.py


In [20]:
rest_rec_bp = si.bandpass_filter(rest_rec, freq_min=300, freq_max=6000) # apply filter
folder_SC = out_path / 'Phy/SC-REST'
folder_SC_wv = folder_SC / 'waveforms_filtered_new'
we_SC = si.extract_waveforms(rest_rec_bp, SC, folder_SC_wv,
    load_if_exists=True, ms_before=1, ms_after=2., max_spikes_per_unit=500,
    chunk_size=30000, n_jobs=6, progress_bar=True)
print(we_SC)

extract waveforms: 100%|██████████| 501/501 [04:25<00:00,  1.88it/s]

WaveformExtractor: 384 channels - 325 units - 1 segments
  before:30 after60 n_per_units: 500


In [21]:
folder_SC_params = out_path / 'Phy/SC-REST/phy-params_new'
si.export_to_phy(rest_rec_bp, SC, folder_SC_params, waveform_extractor=we_SC,
            compute_pc_features=False, compute_amplitudes=True,
            remove_if_exists=True, chunk_size=30000, n_jobs=6, progress_bar=True)

You can run spikeinterface.toolkit.preprocessing.bandpass_filter(recording)
write_binary_recording with n_jobs 6  chunk_size 30000


extract amplitudes: 100%|██████████| 501/501 [00:40<00:00, 12.50it/s]

Run:
phy template-gui  /media/storage/spikesorting_output/REST/Phy/SC-REST/phy-params_new/params.py


## compare all output

In [22]:
comp = si.compare_two_sorters(sorting_SC, sorting_TDC, sorting1_name='SC', sorting2_name='TDC')
si.plot_agreement_matrix(comp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
comp.match_event_count

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,12,19,8,9,19,6,1,8,9,11,5,0,11,3,1
1,28,65,20,26,26,18,1,30,13,22,4,1,19,4,6
10,19,69,10,27,40,24,0,21,15,24,3,4,13,3,3
100,6,11,2,5,5,3,0,7,2,2,0,0,1,1,0
101,20,50,11,12,19,10,0,21,8,17,4,1,19,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,17,31,12,14,28,15,0,11,13,13,0,2,10,6,4
96,29,105,18,28,53,21,1,29,21,27,0,3,13,5,5
97,6,19,1,6,8,8,0,8,4,7,1,0,1,1,1
98,29,66,13,22,37,20,1,24,15,22,2,0,15,5,4


In [24]:
comp.agreement_scores

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.002940,0.001884,0.002365,0.002460,0.003027,0.001676,0.000686,0.001972,0.003054,0.002784,0.002843,0.000000,0.002944,0.001596,0.000424
1,0.005055,0.005646,0.004129,0.005083,0.003358,0.003569,0.000341,0.005447,0.002943,0.004064,0.001237,0.000331,0.003652,0.001193,0.001568
10,0.003065,0.005674,0.001816,0.004683,0.004773,0.004219,0.000000,0.003404,0.002960,0.003958,0.000772,0.001090,0.002218,0.000749,0.000669
100,0.001980,0.001217,0.000858,0.001919,0.000955,0.001187,0.000000,0.002333,0.001054,0.000689,0.000000,0.000000,0.000372,0.001212,0.000000
101,0.004064,0.004587,0.002602,0.002665,0.002667,0.002260,0.000000,0.004294,0.002107,0.003547,0.001534,0.000418,0.004151,0.001834,0.001562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.003626,0.002901,0.003007,0.003282,0.004070,0.003584,0.000000,0.002357,0.003656,0.002850,0.000000,0.000926,0.002298,0.002410,0.001348
96,0.004645,0.008623,0.003243,0.004813,0.006294,0.003655,0.000275,0.004667,0.004106,0.004416,0.000000,0.000806,0.002198,0.001232,0.001103
97,0.001762,0.002021,0.000369,0.002013,0.001427,0.002761,0.000000,0.002370,0.001761,0.002138,0.000924,0.000000,0.000326,0.000833,0.000596
98,0.004608,0.005380,0.002319,0.003745,0.004359,0.003450,0.000271,0.003828,0.002901,0.003566,0.000501,0.000000,0.002516,0.001217,0.000872


## some visualizations - TDC

In [25]:
si.plot_rasters(TDC)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
TDC_units_ID = TDC.unit_ids[:] # I couldnt use just unit 2, was giving me an into has not length error (see below)
si.plot_unit_waveforms(we_TDC, 
                       #channel_ids=TDC_units_ID
                       max_channels=12, 
                       #radius_um=60
                      )

array([  0,   1,  10, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
        11, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,  12, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129,  13, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 139,  14, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149,  15, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159,  16, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169,  17, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179,  18,
       180, 181, 182, 183, 184, 185, 186, 187, 188, 189,  19, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199,   2,  20, 200, 201, 202,
       203, 204, 205, 206, 207, 208, 209,  21, 210, 211, 212, 213, 214,
       215, 216, 217, 218, 219,  22, 220, 221, 222, 223, 224, 225, 226,
       227, 228, 229,  23, 230, 231, 232, 233, 234, 235, 236, 237, 238,
       239,  24, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249,  25,
       250, 251, 252, 253, 254, 255, 256, 257, 258, 259,  26, 26

In [26]:
si.plot_isi_distribution(TDC, 
                         bins=100, 
                         window=1
                        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
colors = {
    0: 'black',
    1: 'green',
    2: 'red',
    3: 'blue',
    4: 'yellow',
    5: 'grey',
    6: 'purple',
    7: 'orange',
    8: 'lightblue'}
w_depth = si.plot_units_depth_vs_amplitude(we_TDC,
                                          #unit_colors=colors
                                          )
w_depth.ax.set_aspect(0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
si.plot_amplitudes_distribution(we_TDC)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 ## some visualizations - SC (heavier so won't plot everything) 

In [ ]:
w_depth_SC = si.plot_units_depth_vs_amplitude(we_SC)
w_depth_SC.ax.set_aspect(0.1)